In [1]:
# -*- coding: utf-8 -*-
#==========================================
# Title:  BaseBO.py
# Author: Binxin Ru and Ahsan Alvi
# Date:   20 August 2019
# Link:   https://arxiv.org/abs/1906.08878
#==========================================

import os
import pickle
import random

import numpy as np


class BaseBO():
    """
    Base class with common operations for BO with continuous and categorical
    inputs
    """

    def __init__(self, objfn, initN, bounds, C, rand_seed=108, debug=False,
                 batch_size=1, **kwargs):
        self.f = objfn  # function to optimise
        self.bounds = bounds  # function bounds
        self.batch_size = batch_size
        self.C = C  # no of categories
        self.initN = initN  # no: of initial points
        self.nDim = len(self.bounds)  # dimension
        self.rand_seed = rand_seed
        self.debug = debug
        self.saving_path = None
        self.kwargs = kwargs
        self.x_bounds = np.vstack([d['domain'] for d in self.bounds
                                   if d['type'] == 'continuous'])

    def initialise(self, seed):
        """Get NxN intial points"""
        data = []
        result = []

        np.random.seed(seed)
        random.seed(seed)

        init_fname = self.saving_path + 'init_data_' + str(seed)

        if os.path.exists(init_fname):
            print(f"Using existing init data for seed {seed}")
            with open(init_fname, 'rb') as init_data_filefile2:
                init_data = pickle.load(init_data_filefile2)
            Zinit = init_data['Z_init']
            yinit = init_data['y_init']
        else:
            print(f"Creating init data for seed {seed}")
            Xinit = self.generateInitialPoints(self.initN,
                                               self.bounds[len(self.C):])
            hinit = np.hstack(
                [np.random.randint(0, C, self.initN)[:, None] for C in self.C])
            Zinit = np.hstack((hinit, Xinit))
            yinit = np.zeros([Zinit.shape[0], 1])

            for j in range(self.initN):
                ht_list = list(hinit[j])
                yinit[j] = self.f(ht_list, Xinit[j])
                # print(ht_list, Xinit[j], yinit[j])

            init_data = {}
            init_data['Z_init'] = Zinit
            init_data['y_init'] = yinit

            with open(init_fname, 'wb') as init_data_file:
                pickle.dump(init_data, init_data_file)

        data.append(Zinit)
        result.append(yinit)
        return data, result

    def generateInitialPoints(self, initN, bounds):
        nDim = len(bounds)
        Xinit = np.zeros((initN, len(bounds)))
        for i in range(initN):
            Xinit[i, :] = np.array(
                [np.random.uniform(bounds[b]['domain'][0],
                                   bounds[b]['domain'][1], 1)[0]
                 for b in range(nDim)])
        return Xinit

    def my_func(self, Z):
        Z = np.atleast_2d(Z)
        if len(Z) == 1:
            X = Z[0, len(self.C):]
            ht_list = list(Z[0, :len(self.C)])
            return self.f(ht_list, X)
        else:
            f_vals = np.zeros(len(Z))
            for ii in range(len(Z)):
                X = Z[ii, len(self.C):]
                ht_list = list(Z[ii, :len(self.C)].astype(int))
                f_vals[ii] = self.f(ht_list, X)
            return f_vals

    def save_progress_to_disk(self, *args):
        raise NotImplementedError

    def runTrials(self, trials, budget, saving_path):
        raise NotImplementedError


In [9]:
# -*- coding: utf-8 -*-
#==========================================
# Title:  BatchCoCaBO.py
# Author: Binxin Ru and Ahsan Alvi
# Date:   20 August 2019
# Link:   https://arxiv.org/abs/1906.08878
#==========================================

import math

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils.bayesopt.batch_bo import BatchBOHeuristic
from utils.bayesopt.executor import JobExecutorInSeriesBlocking
from utils.bayesopt.util import add_hallucinations_to_x_and_y
from methods.CoCaBO_Base import CoCaBO_Base
from utils.ml_utils.models import GP
from utils.ml_utils.models.additive_gp import MixtureViaSumAndProduct, \
    CategoryOverlapKernel, GPWithSomeFixedDimsAtStart


''' Batch CoCaBO algorithm '''
class BatchCoCaBO(CoCaBO_Base):

    def __init__(self, objfn, initN, bounds, acq_type, C, **kwargs):

        super(BatchCoCaBO, self).__init__(objfn, initN, bounds, acq_type, C,
                                          **kwargs)
        self.best_val_list = []
        self.C_list = self.C
        self.name = 'BCoCaBO'

    def runOptim(self, budget, seed, initData=None,
                 initResult=None, ):

        if (initData and initResult):
            self.data = initData[:]
            self.result = initResult[:]
        else:
            self.data, self.result = self.initialise(seed)

        bestUpperBoundEstimate = 2 * budget / 3

        gamma_list = [np.sqrt(C * math.log(C / self.batch_size) / (
                (math.e - 1) * self.batch_size * bestUpperBoundEstimate))
                      for C in self.C_list]
        gamma_list = [g if not np.isnan(g) else 1 for g in gamma_list]

        Wc_list_init = [np.ones(C) for C in self.C_list]
        Wc_list = Wc_list_init
        nDim = len(self.bounds)

        result_list = []
        starting_best = np.max(-1 * self.result[0])
        result_list.append([-1, None, None, starting_best, None])

        continuous_dims = list(range(len(self.C_list), nDim))
        categorical_dims = list(range(len(self.C_list)))

        for t in tqdm(range(budget)):
            self.iteration = t
            ht_batch_list, probabilityDistribution_list, S0 = self.compute_prob_dist_and_draw_hts(
                Wc_list, gamma_list, self.batch_size)

            ht_batch_list = ht_batch_list.astype(int)

            # Obtain the reward for multi-armed bandit: B x len(self.C_list)
            Gt_ht_list = self.RewardperCategoryviaBO(self.f, ht_batch_list,
                                                     categorical_dims,
                                                     continuous_dims)

            # Update the reward and the weight
            Wc_list = self.update_weights_for_all_cat_var(Gt_ht_list,
                                                          ht_batch_list,
                                                          Wc_list, gamma_list,
                                                          probabilityDistribution_list,
                                                          self.batch_size,
                                                          S0=S0)

            # Get the best value till now
            besty, li, vi = self.getBestVal2(self.result)

            # Store the results of this iteration
            result_list.append(
                [t, ht_batch_list, Gt_ht_list, besty, self.mix_used,
                 self.model_hp])
            self.ht_recommedations.append(ht_batch_list)

        df = pd.DataFrame(result_list,
                          columns=["iter", "ht", "Reward", "best_value",
                                   "mix_val", "model_hp"])
        bestx = self.data[li][vi]
        self.best_val_list.append(
            [self.batch_size, self.trial_num, li, besty, bestx])
        return df

    # =============================================================================
    #   Function returns the reward for multi-armed bandit
    # =============================================================================
    def RewardperCategoryviaBO(self, objfn, ht_next_batch_list,
                               categorical_dims,
                               continuous_dims):

        #  Get observation data
        Zt = self.data[0]
        yt = self.result[0]

        my_kernel, hp_bounds = self.get_kernel(categorical_dims,
                                               continuous_dims)

        gp_opt_params = {'method': 'multigrad',
                         'num_restarts': 5,
                         'restart_bounds': hp_bounds,
                         'hp_bounds': hp_bounds,
                         'verbose': False}

        gp_kwargs = {'y_norm': 'meanstd',
                     'opt_params': gp_opt_params}
        gp_args = (Zt, yt, my_kernel)

        gp = GP(*gp_args, **gp_kwargs)

        opt_flag, gp = self.set_model_params_and_opt_flag(gp)
        if opt_flag:
            # print("\noptimising!\n")
            gp.optimize()
        self.model_hp = gp.param_array

        acq_dict = {'type': 'subspace'}

        acq_opt_params = {'method': 'samplegrad',
                          'num_local': 5,
                          'num_samples': 5000,
                          'num_chunks': 10,
                          'verbose': False}

        ymin_opt_params = {'method': 'standard'}

        # Find the unique combinations in h and their frequency
        h_unique, h_counts = np.unique(ht_next_batch_list,
                                       return_counts=True, axis=0)

        # Create the batch
        z_batch_list = []
        for idx, curr_h in enumerate(h_unique):
            # Perform batch BO with a single fixed h
            gp_for_bo = GPWithSomeFixedDimsAtStart(*gp_args,
                                                   fixed_dim_vals=curr_h,
                                                   **gp_kwargs)
            gp_for_bo.param_array = gp.param_array

            curr_batch_size = h_counts[idx]
            interface = JobExecutorInSeriesBlocking(curr_batch_size)

            # Adding repulsion effect to already-selected locations
            if len(z_batch_list) > 0:
                self.surrogate = gp_for_bo
                self.async_infill_strategy = 'kriging_believer'  # hack
                surrogate_x_with_fake, surrogate_y_with_fake = \
                    add_hallucinations_to_x_and_y(
                        self, gp_for_bo.X, gp_for_bo.Y_raw,
                        np.vstack(z_batch_list))
                gp_for_bo.set_XY(X=surrogate_x_with_fake,
                                 Y=surrogate_y_with_fake)

            bo = BatchBOHeuristic(objfn, gp_for_bo, self.x_bounds,
                                  async_infill_strategy='kriging_believer',
                                  offset_acq=True,
                                  async_interface=interface,
                                  batch_size=curr_batch_size,
                                  acq_dict=acq_dict,
                                  y_min_opt_params=ymin_opt_params,
                                  acq_opt_params=acq_opt_params,
                                  optimise_surrogate_model=False)

            x_batch_for_curr_h, _ = bo.get_next()

            z_batch_for_curr_h = np.hstack((
                np.vstack([curr_h] * curr_batch_size),
                np.vstack(x_batch_for_curr_h)
            ))
            z_batch_list.append(z_batch_for_curr_h)

        z_batch_next = np.vstack(z_batch_list)

        #  Evaluate objective function at
        y_batch_next = np.zeros((self.batch_size, 1))
        for b in range(self.batch_size):
            x_next = z_batch_next[b, continuous_dims]
            ht_next_list = z_batch_next[b, categorical_dims]
            try:
                y_next = objfn(ht_next_list, x_next)
            except:
                print('stop')

            y_batch_next[b] = y_next

        # Append recommeded data
        self.mix_used = gp.kern.mix[0]
        self.data[0] = np.row_stack((self.data[0], z_batch_next))
        self.result[0] = np.row_stack((self.result[0], y_batch_next))

        # Obtain the reward for each categorical variable: B x len(self.C_list)
        ht_batch_list_rewards = self.compute_reward_for_all_cat_variable(
            ht_next_batch_list, self.batch_size)

        bestval_ht = np.max(self.result[0] * -1)
        # print(f'arm pulled={ht_next_batch_list[:]} ; '
        #       f'\n rewards = {ht_batch_list_rewards[:]}; '
        #       f'y_best = {bestval_ht}; mix={self.mix_used}')
        print(f'arm pulled={ht_next_batch_list[:]} ; '
              f'y_best = {bestval_ht}; mix={self.mix_used}')

        return ht_batch_list_rewards

    def get_kernel(self, categorical_dims, continuous_dims):
        # Create surrogate model
        if self.ARD:
            hp_bounds = np.array([
                *[[1e-4, 3]] * len(continuous_dims),  # cont lengthscale
                [1e-6, 1],  # likelihood variance
            ])
        else:
            hp_bounds = np.array([
                [1e-4, 3],  # cont lengthscale
                [1e-6, 1],  # likelihood variance
            ])
        fix_mix_in_this_iter, mix_value, hp_bounds = self.get_mix(hp_bounds)
        k_cat = CategoryOverlapKernel(len(categorical_dims),
                                      active_dims=categorical_dims)  # cat
        k_cont = GPy.kern.Matern52(len(continuous_dims),
                                   lengthscale=self.default_cont_lengthscale,
                                   active_dims=continuous_dims,
                                   ARD=self.ARD)  # cont
        my_kernel = MixtureViaSumAndProduct(
            len(categorical_dims) + len(continuous_dims),
            k_cat, k_cont, mix=mix_value, fix_inner_variances=True,
            fix_mix=fix_mix_in_this_iter)
        return my_kernel, hp_bounds


In [3]:
# -*- coding: utf-8 -*-
#==========================================
# Title:  CoCaBO_Base.py
# Author: Binxin Ru and Ahsan Alvi
# Date:   20 August 2019
# Link:   https://arxiv.org/abs/1906.08878
#==========================================

import collections
import pickle
import random

import numpy as np
from scipy.optimize import minimize

from methods.BaseBO import BaseBO
from utils.DepRound import DepRound
from utils.probability import distr, draw


class CoCaBO_Base(BaseBO):

    def __init__(self, objfn, initN, bounds, acq_type, C,
                 kernel_mix=0.5, mix_lr=10,
                 model_update_interval=10,
                 ard=False, **kwargs):
        super().__init__(objfn, initN, bounds, C, **kwargs)
        self.acq_type = acq_type

        # Store the ht recommendations for each iteration
        self.ht_recommedations = []
        self.ht_hist_batch = []

        # Store the name of the algorithm
        self.policy = None

        self.X = []
        self.Y = []

        # To check the best vals
        self.gp_bestvals = []

        self.ARD = ard

        # Keeping track of current iteration helps control mix learning
        self.iteration = None

        self.model_hp = None
        self.default_cont_lengthscale = 0.2

        self.mix = kernel_mix
        if ((model_update_interval % mix_lr == 0) or
                (mix_lr % model_update_interval == 0)):
            self.mix_learn_rate = mix_lr
            self.model_update_interval = model_update_interval
        else:
            self.mix_learn_rate = min(mix_lr, model_update_interval)
            self.model_update_interval = min(mix_lr, model_update_interval)
        self.mix_used = 0.5

        self.name = None

    def estimate_alpha(self, batch_size, gamma, Wc, C):

        def single_evaluation(alpha):
            denominator = sum([alpha if val > alpha else val for idx, val in enumerate(Wc)])
            rightside = (1 / batch_size - gamma / C) / (1 - gamma)
            output = np.abs(alpha / denominator - rightside)

            return output

        x_tries = np.random.uniform(0, np.max(Wc), size=(100, 1))
        y_tries = [single_evaluation(val) for val in x_tries]
        # find x optimal for init
        # print(f'ytry_len={len(y_tries)}')
        idx_min = np.argmin(y_tries)
        x_init_min = x_tries[idx_min]

        res = minimize(single_evaluation, x_init_min, method='BFGS', options={'gtol': 1e-6, 'disp': False})
        if isinstance(res, float):
            return res
        else:
            return res.x

    def runTrials(self, trials, budget, saving_path):
        # Initialize mean_bestvals, stderr, hist
        best_vals = []
        mix_values = []
        debug_values = []
        n_working = trials
        self.saving_path = saving_path

        for i in range(trials):
            print("Running trial: ", i)
            self.trial_num = i
            np.random.seed(i)
            random.seed(i)

            df = self.runOptim(budget=budget, seed=i)
            best_vals.append(df['best_value'])
            mix_values.append(df['mix_val'])
            self.save_progress_to_disk(best_vals, debug_values, mix_values,
                                       saving_path, df)

        # Runoptim updates the ht_recommendation histogram
        self.best_vals = best_vals
        ht_hist = collections.Counter(np.array(self.ht_recommedations).ravel())
        self.ht_recommedations = []
        self.mean_best_vals = np.mean(best_vals, axis=0)
        self.err_best_vals = np.std(best_vals, axis=0) / np.sqrt(n_working)

        # For debugging
        self.gp_bestvals = best_vals
        self.ht_hist = ht_hist
        self.n_working = n_working
        return self.mean_best_vals, self.err_best_vals, ht_hist

    def save_progress_to_disk(self, best_vals, debug_values, mix_values,
                              saving_path, df):
        results_file_name = saving_path + self.name + \
                            f'_{self.batch_size}' + \
                            '_best_vals_' + \
                            self.acq_type + \
                            '_ARD_' + str(self.ARD) + '_mix_' + \
                            str(self.mix)

        with open(results_file_name, 'wb') as file:
            pickle.dump(best_vals, file)
        if self.mix > 1 or self.mix < 0:
            mix_file_name = saving_path + self.name + \
                            f'_{self.batch_size}_' + \
                            self.acq_type + \
                            '_ARD_' + str(self.ARD) + '_mix_' + \
                            str(self.mix) + '_mix_values'
            with open(mix_file_name, 'wb') as file2:
                pickle.dump(mix_values, file2)
        if self.debug:
            debug_file_name = saving_path + self.name + \
                              f'_{self.batch_size}_' + \
                              self.acq_type + \
                              '_ARD_' + str(self.ARD) + '_mix_' + \
                              str(self.mix) + '_debug'
            with open(debug_file_name, 'wb') as file2:
                pickle.dump(debug_values, file2)

        df.to_pickle(f"{results_file_name}_df_s{self.trial_num}")

    def compute_reward_for_all_cat_variable(self, ht_next_batch_list, batch_size):
        # Obtain the reward for each categorical variable: B x len(self.C_list)
        ht_batch_list_rewards = np.zeros((batch_size, len(self.C_list)))
        for b in range(batch_size):
            ht_next_list = ht_next_batch_list[b, :]

            for i in range(len(ht_next_list)):
                idices = np.where(self.data[0][:, i] == ht_next_list[i])
                ht_result = self.result[0][idices]
                ht_reward = np.max(ht_result * -1)
                ht_batch_list_rewards[b, i] = ht_reward
        return ht_batch_list_rewards

    def update_weights_for_all_cat_var(self, Gt_ht_list, ht_batch_list, Wc_list, gamma_list,
                                       probabilityDistribution_list, batch_size, S0=None):
        for j in range(len(self.C_list)):
            Wc = Wc_list[j]
            C = self.C_list[j]
            gamma = gamma_list[j]
            probabilityDistribution = probabilityDistribution_list[j]
            # print(f'cat_var={j}, prob={probabilityDistribution}')

            if batch_size > 1:
                ht_batch_list = ht_batch_list.astype(int)
                Gt_ht = Gt_ht_list[:, j]
                mybatch_ht = ht_batch_list[:, j]  # 1xB
                for ii, ht in enumerate(mybatch_ht):
                    Gt_ht_b = Gt_ht[ii]
                    estimatedReward = 1.0 * Gt_ht_b / probabilityDistribution[ht]
                    if ht not in S0:
                        Wc[ht] *= np.exp(batch_size * estimatedReward * gamma / C)
            else:
                Gt_ht = Gt_ht_list[j]
                ht = ht_batch_list[j]  # 1xB
                estimatedReward = 1.0 * Gt_ht / probabilityDistribution[ht]
                Wc[ht] *= np.exp(estimatedReward * gamma / C)

        return Wc_list

    def compute_prob_dist_and_draw_hts(self, Wc_list, gamma_list, batch_size):

        if batch_size > 1:
            ht_batch_list = np.zeros((batch_size, len(self.C_list)))
            probabilityDistribution_list = []

            for j in range(len(self.C_list)):
                Wc = Wc_list[j]
                gamma = gamma_list[j]
                C = self.C_list[j]
                # perform some truncation here
                maxW = np.max(Wc)
                temp = np.sum(Wc) * (1.0 / batch_size - gamma / C) / (1 - gamma)
                if gamma < 1 and maxW >= temp:
                    # find a threshold alpha
                    alpha = self.estimate_alpha(batch_size, gamma, Wc, C)
                    S0 = [idx for idx, val in enumerate(Wc) if val > alpha]
                else:
                    S0 = []
                # Compute the probability for each category
                probabilityDistribution = distr(Wc, gamma)

                # draw a batch here
                if batch_size < C:
                    mybatch_ht = DepRound(probabilityDistribution, k=batch_size)
                else:
                    mybatch_ht = np.random.choice(len(probabilityDistribution), batch_size, p=probabilityDistribution)

                # ht_batch_list size: len(self.C_list) x B
                ht_batch_list[:, j] = mybatch_ht[:]

                # ht_batch_list.append(mybatch_ht)
                probabilityDistribution_list.append(probabilityDistribution)

            return ht_batch_list, probabilityDistribution_list, S0

        else:
            ht_list = []
            probabilityDistribution_list = []
            for j in range(len(self.C_list)):
                Wc = Wc_list[j]
                gamma = gamma_list[j]
                # Compute the probability for each category
                probabilityDistribution = distr(Wc, gamma)
                # Choose a categorical variable at random
                ht = draw(probabilityDistribution)
                ht_list.append(ht)
                probabilityDistribution_list.append(probabilityDistribution)

            return ht_list, probabilityDistribution_list

    def compute_weights_for_init_data(self, Wc_list_init, gamma_list, batch_size):
        ht_next_batch_list = self.data[0][:, :len(self.C)]
        _, probabilityDistribution_list, S0 = self.compute_prob_dist_and_draw_hts(Wc_list_init, gamma_list,
                                                                                  ht_next_batch_list.shape[0])
        Gt_ht_list = self.compute_reward_for_all_cat_variable(ht_next_batch_list, ht_next_batch_list.shape[0])
        New_Wc_list = self.update_weights_for_all_cat_var(Gt_ht_list, ht_next_batch_list, Wc_list_init, gamma_list,
                                                          probabilityDistribution_list, ht_next_batch_list.shape[0],
                                                          S0=S0)

        return New_Wc_list

    def get_mix(self, hp_bounds):
        fix_mix_in_this_iter = True
        if (self.mix >= 0) and (self.mix <= 1):  # mix param is fixed
            mix_value = self.mix
        elif ((self.iteration >= self.mix_learn_rate) and
              (self.iteration % self.mix_learn_rate == 0)):
            # learn mix
            hp_bounds = np.vstack(([1e-6, 1], hp_bounds))
            fix_mix_in_this_iter = False
            mix_value = 0.5
        else:  # between learning iterations
            mix_value = self.mix_used
        return fix_mix_in_this_iter, mix_value, hp_bounds

    # ========================================
    #     Over-ride this!
    # =============================================================================
    def runOptim(self, budget, seed):
        raise NotImplementedError

    # =============================================================================
    # Get best value from nested list along with the index
    # =============================================================================
    def getBestVal2(self, my_list):
        temp = [np.max(i * -1) for i in my_list]
        indx1 = [np.argmax(i * -1) for i in my_list]
        indx2 = np.argmax(temp)
        val = np.max(temp)
        list_indx = indx2
        val_indx = indx1[indx2]
        return val, list_indx, val_indx

    def set_model_params_and_opt_flag(self, model):
        """
        Returns opt_flag, model
        """
        if ((self.iteration >= self.model_update_interval) and
                (self.iteration % self.model_update_interval == 0)):
            return True, model
        else:
            # No previous model_hp, so optimise
            if self.model_hp is None:
                self.model_hp = model.param_array
            else:
                # print(self.model_hp)
                # print(model.param_array)
                # previous iter learned mix, so remove mix before setting
                if len(model.param_array) < len(self.model_hp):
                    model.param_array = self.model_hp[1:]
                else:
                    model.param_array = self.model_hp

            return False, model


In [4]:
# -*- coding: utf-8 -*-
#==========================================
# Title:  CoCaBO.py
# Author: Binxin Ru and Ahsan Alvi
# Date:   20 August 2019
# Link:   https://arxiv.org/abs/1906.08878
#==========================================

import math

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils.bayesopt.acquisition import AcquisitionOnSubspace, EI, UCB
from methods.CoCaBO_Base import CoCaBO_Base
from utils.ml_utils.models import GP
from utils.ml_utils.models.additive_gp import MixtureViaSumAndProduct, \
    CategoryOverlapKernel
from utils.ml_utils.optimization import sample_then_minimize

''' Sequential CoCaBO algorithm '''
class CoCaBO(CoCaBO_Base):

    def __init__(self, objfn, initN, bounds, acq_type, C, **kwargs):

        super(CoCaBO, self).__init__(objfn, initN, bounds, acq_type, C, **kwargs)
        self.best_val_list = []
        self.C_list = self.C
        self.name = 'CoCaBO'

    def runOptim(self, budget, seed, batch_size=1, initData=None, initResult=None):

        if (initData and initResult):
            self.data = initData[:]
            self.result = initResult[:]
        else:
            self.data, self.result = self.initialise(seed)

        # Initialize wts and probs
        b = batch_size
        bestUpperBoundEstimate = 2 * budget / 3
        gamma_list = [math.sqrt(C * math.log(C) /
                                ((math.e - 1) * bestUpperBoundEstimate))
                      for C in self.C_list]
        Wc_list_init = [np.ones(C) for C in self.C_list]
        Wc_list = Wc_list_init
        nDim = len(self.bounds)

        result_list = []
        starting_best = np.max(-1 * self.result[0])
        result_list.append([-1, None, None, starting_best, None])

        continuous_dims = list(range(len(self.C_list), nDim))
        categorical_dims = list(range(len(self.C_list)))

        for t in tqdm(range(budget)):
            self.iteration = t

            # Compute the probability for each category and Choose categorical variables
            ht_list, probabilityDistribution_list = \
                self.compute_prob_dist_and_draw_hts(Wc_list, gamma_list,
                                                    batch_size)

            # Get reward for multi-armed bandit
            Gt_ht_list = self.RewardperCategoryviaBO(self.f, ht_list,
                                                     categorical_dims,
                                                     continuous_dims,
                                                     self.bounds,
                                                     self.acq_type, b)

            # Update the reward and the weight
            Wc_list = self.update_weights_for_all_cat_var(
                Gt_ht_list, ht_list,
                Wc_list, gamma_list,
                probabilityDistribution_list,
                batch_size)

            # Get the best value till now
            besty, li, vi = self.getBestVal2(self.result)

            # Store the results of this iteration
            result_list.append([t, ht_list, Gt_ht_list, besty, self.mix_used,
                                self.model_hp])

            self.ht_recommedations.append(ht_list)

        df = pd.DataFrame(result_list, columns=["iter", "ht", "Reward",
                                                "best_value", "mix_val",
                                                "model_hp"])
        bestx = self.data[li][vi]
        self.best_val_list.append([batch_size, self.trial_num, li, besty,
                                   bestx])

        return df

    # =============================================================================
    #   Function returns the reward for multi-armed bandit
    # =============================================================================
    def RewardperCategoryviaBO(self, objfn, ht_next_list, categorical_dims,
                               continuous_dims, bounds, acq_type, b):

        #  Get observation data
        Zt = self.data[0]
        yt = self.result[0]

        my_kernel, hp_bounds = self.get_kernel(categorical_dims,
                                               continuous_dims)

        gp_opt_params = {'method': 'multigrad',
                         'num_restarts': 5,
                         'restart_bounds': hp_bounds,
                         'hp_bounds': hp_bounds,
                         'verbose': False}

        gp = GP(Zt, yt, my_kernel, y_norm='meanstd',
                opt_params=gp_opt_params)

        opt_flag, gp = self.set_model_params_and_opt_flag(gp)
        if opt_flag:
            # print("\noptimising!\n")
            gp.optimize()
        self.model_hp = gp.param_array


        self.mix_used = gp.kern.mix[0]

        x_bounds = np.array([d['domain'] for d in bounds
                             if d['type'] == 'continuous'])
        # create acq
        if acq_type == 'EI':
            acq = EI(gp, np.min(gp.Y_raw))
        elif acq_type == 'LCB':
            acq = UCB(gp, 2.0)

        acq_sub = AcquisitionOnSubspace(acq, my_kernel.k2.active_dims,
                                        ht_next_list)

        def optimiser_func(x):
            return -acq_sub.evaluate(np.atleast_2d(x))

        res = sample_then_minimize(
            optimiser_func,
            x_bounds,
            num_samples=5000,
            num_chunks=10,
            num_local=3,
            minimize_options=None,
            evaluate_sequentially=False)

        x_next = res.x
        z_next = np.hstack((ht_next_list, x_next))

        #  Evaluate objective function at z_next = [x_next,  ht_next_list]
        y_next = objfn(ht_next_list, x_next)

        # Append recommeded data
        self.data[0] = np.row_stack((self.data[0], z_next))
        self.result[0] = np.row_stack((self.result[0], y_next))

        # Obtain the reward for each categorical variable
        ht_next_list_array = np.atleast_2d(ht_next_list)
        ht_list_rewards = self.compute_reward_for_all_cat_variable(
            ht_next_list_array, b)
        ht_list_rewards = list(ht_list_rewards.flatten())

        bestval_ht = np.max(self.result[0] * -1)
        # print(f'arm pulled={ht_next_list[:]} ; rewards = {ht_list_rewards[:]};'
        #       f' y_best = {bestval_ht}; mix={self.mix_used}')
        print(f'arm pulled={ht_next_list[:]}; y_best = {bestval_ht}; mix={self.mix_used}')

        return ht_list_rewards

    def get_kernel(self, categorical_dims, continuous_dims):
        # create surrogate
        if self.ARD:
            hp_bounds = np.array([
                *[[1e-4, 3]] * len(continuous_dims),  # cont lengthscale
                [1e-6, 1],  # likelihood variance
            ])
        else:
            hp_bounds = np.array([
                [1e-4, 3],  # cont lengthscale
                [1e-6, 1],  # likelihood variance
            ])
        fix_mix_in_this_iter, mix_value, hp_bounds = self.get_mix(hp_bounds)
        k_cat = CategoryOverlapKernel(len(categorical_dims),
                                      active_dims=categorical_dims)  # cat
        k_cont = GPy.kern.Matern52(len(continuous_dims),
                                   lengthscale=self.default_cont_lengthscale,
                                   active_dims=continuous_dims,
                                   ARD=self.ARD)  # cont
        my_kernel = MixtureViaSumAndProduct(
            len(categorical_dims) + len(continuous_dims),
            k_cat, k_cont, mix=mix_value, fix_inner_variances=True,
            fix_mix=fix_mix_in_this_iter)
        return my_kernel, hp_bounds


In [6]:
# -*- coding: utf-8 -*-
#==========================================
# Title:  syntheticFunctions.py
# Author: Binxin Ru and Ahsan Alvi
# Date:   20 August 2019
# Link:   https://arxiv.org/abs/1906.08878
#==========================================

import numpy as np

# =============================================================================
# Rosenbrock Function (f_min = 0)
# https://www.sfu.ca/~ssurjano/rosen.html
# =============================================================================
def myrosenbrock(X):
    X = np.asarray(X)
    X = X.reshape((-1, 2))
    if len(X.shape) == 1:  # one observation
        x1 = X[0]
        x2 = X[1]
    else:  # multiple observations
        x1 = X[:, 0]
        x2 = X[:, 1]
    fx = 100 * (x2 - x1 ** 2) ** 2 + (x1 - 1) ** 2
    return fx.reshape(-1, 1) / 300

# =============================================================================
#  Six-hump Camel Function (f_min = - 1.0316 )
#  https://www.sfu.ca/~ssurjano/camel6.html       
# =============================================================================
def mysixhumpcamp(X):
    X = np.asarray(X)
    X = np.reshape(X, (-1, 2))
    if len(X.shape) == 1:
        x1 = X[0]
        x2 = X[1]
    else:
        x1 = X[:, 0]
        x2 = X[:, 1]
    term1 = (4 - 2.1 * x1 ** 2 + (x1 ** 4) / 3) * x1 ** 2
    term2 = x1 * x2
    term3 = (-4 + 4 * x2 ** 2) * x2 ** 2
    fval = term1 + term2 + term3
    return fval.reshape(-1, 1) / 10

# =============================================================================
# Beale function (f_min = 0)
# https://www.sfu.ca/~ssurjano/beale.html
# =============================================================================
def mybeale(X):
    X = np.asarray(X) / 2
    X = X.reshape((-1, 2))
    if len(X.shape) == 1:
        x1 = X[0] * 2
        x2 = X[1] * 2
    else:
        x1 = X[:, 0] * 2
        x2 = X[:, 1] * 2
    fval = (1.5 - x1 + x1 * x2) ** 2 + (2.25 - x1 + x1 * x2 ** 2) ** 2 + (
            2.625 - x1 + x1 * x2 ** 3) ** 2
    return fval.reshape(-1, 1) / 50


def func2C(ht_list, X):
    # ht is a categorical index
    # X is a continuous variable
    X = X * 2

    assert len(ht_list) == 2
    ht1 = ht_list[0]
    ht2 = ht_list[1]

    if ht1 == 0:  # rosenbrock
        f = myrosenbrock(X)
    elif ht1 == 1:  # six hump
        f = mysixhumpcamp(X)
    elif ht1 == 2:  # beale
        f = mybeale(X)

    if ht2 == 0:  # rosenbrock
        f = f + myrosenbrock(X)
    elif ht2 == 1:  # six hump
        f = f + mysixhumpcamp(X)
    else:
        f = f + mybeale(X)

    y = f + 1e-6 * np.random.rand(f.shape[0], f.shape[1])
    return y.astype(float)


def func3C(ht_list, X):
    # ht is a categorical index
    # X is a continuous variable
    X = np.atleast_2d(X)
    assert len(ht_list) == 3
    ht1 = ht_list[0]
    ht2 = ht_list[1]
    ht3 = ht_list[2]

    X = X * 2
    if ht1 == 0:  # rosenbrock
        f = myrosenbrock(X)
    elif ht1 == 1:  # six hump
        f = mysixhumpcamp(X)
    elif ht1 == 2:  # beale
        f = mybeale(X)

    if ht2 == 0:  # rosenbrock
        f = f + myrosenbrock(X)
    elif ht2 == 1:  # six hump
        f = f + mysixhumpcamp(X)
    else:
        f = f + mybeale(X)

    if ht3 == 0:  # rosenbrock
        f = f + 5 * mysixhumpcamp(X)
    elif ht3 == 1:  # six hump
        f = f + 2 * myrosenbrock(X)
    else:
        f = f + ht3 * mybeale(X)

    y = f + 1e-6 * np.random.rand(f.shape[0], f.shape[1])

    return y.astype(float)


In [10]:
# -*- coding: utf-8 -*-
#==========================================
# Title:  run_cocabo_exps.py
# Author: Binxin Ru and Ahsan Alvi
# Date:   20 August 2019
# Link:   https://arxiv.org/abs/1906.08878
#==========================================

# =============================================================================
#  CoCaBO Algorithms 
# =============================================================================
import sys
# sys.path.append('../bayesopt')
# sys.path.append('../ml_utils')
import argparse
import os
import testFunctions.syntheticFunctions
from methods.CoCaBO import CoCaBO
from methods.BatchCoCaBO import BatchCoCaBO


def CoCaBO_Exps(obj_func, budget, initN=24 ,trials=40, kernel_mix = 0.5, batch=None):

    # define saving path for saving the results
    saving_path = f'data/syntheticFns/{obj_func}/'
    if not os.path.exists(saving_path):
        os.makedirs(saving_path)

    # define the objective function
    if obj_func == 'func2C':
        f = testFunctions.syntheticFunctions.func2C
        categories = [3, 5]

        bounds = [{'name': 'h1', 'type': 'categorical', 'domain': (0, 1, 2)},
            {'name': 'h2', 'type': 'categorical', 'domain': (0, 1, 2, 3, 4)},
            {'name': 'x1', 'type': 'continuous', 'domain': (-1, 1)},
            {'name': 'x2', 'type': 'continuous', 'domain': (-1, 1)}]

    elif obj_func == 'func3C':
        f = testFunctions.syntheticFunctions.func3C
        categories = [3, 5, 4]

        bounds = [{'name': 'h1', 'type': 'categorical', 'domain': (0, 1, 2)},
            {'name': 'h2', 'type': 'categorical', 'domain': (0, 1, 2, 3, 4)},
            {'name': 'h3', 'type': 'categorical', 'domain': (0, 1, 2, 3)},
            {'name': 'x1', 'type': 'continuous', 'domain': (-1, 1)},
            {'name': 'x2', 'type': 'continuous', 'domain': (-1, 1)}]

    else:
        raise NotImplementedError

    # Run CoCaBO Algorithm
    if batch == 1:
        # sequential CoCaBO
        mabbo = CoCaBO(objfn=f, initN=initN, bounds=bounds,
                       acq_type='LCB', C=categories,
                       kernel_mix = kernel_mix)

    else:
        # batch CoCaBO
        mabbo = BatchCoCaBO(objfn=f, initN=initN, bounds=bounds,
                            acq_type='LCB', C=categories,
                            kernel_mix=kernel_mix,
                            batch_size=batch)
    mabbo.runTrials(trials, budget, saving_path)




if __name__ == '__main__':

    parser = argparse.ArgumentParser(description="Run BayesOpt Experiments")
    parser.add_argument('-f', '--func', help='Objective function',
                        default='func2C', type=str)
    parser.add_argument('-mix', '--kernel_mix',
                        help='Mixture weight for production and summation kernel. Default = 0.0', default=0.0,
                        type=float)
    parser.add_argument('-n', '--max_itr', help='Max Optimisation iterations. Default = 100',
                        default=100, type=int)
    parser.add_argument('-tl', '--trials', help='Number of random trials. Default = 20',
                        default=20, type=int)
    parser.add_argument('-b', '--batch', help='Batch size (>1 for batch CoCaBO and =1 for sequential CoCaBO). Default = 1',
                        default=2, type=int)

    args = parser.parse_args()
    print(f"Got arguments: \n{args}")
    obj_func = args.func
    kernel_mix = args.kernel_mix
    n_itrs = args.max_itr
    n_trials = args.trials
    batch = args.batch

    CoCaBO_Exps(obj_func=obj_func, budget=n_itrs,
                 trials=n_trials, kernel_mix = kernel_mix, batch=batch)


Got arguments: 
Namespace(func='/Users/s/Library/Jupyter/runtime/kernel-v3f138c4f094371e7d174d8c61457d2247b3a6d3d0.json', kernel_mix=0.0, max_itr=100, trials=20, batch=2)


NotImplementedError: 